In [1]:
from SD_IB_IRP_PPenv import steroid_IRP
from Policies import policies

# Policy Evaluation

This Notebook has a complete Policy Evaluation function for the Stochastic-Dynamic Inventory-Routing-Problem with Perishable Products. First, the main parameters of the problem and the environment must be set. All the main customizable parameters are below.

In [2]:
#################################   Environment's parameters   #################################
# SD-IB-IRP-PP model's parameters
horizon_type = 'episodic'
backorders = False
stochastic_parameters = False

# Feature's parameters
look_ahead = ['*']
historical_data = ['*']

# Action's parameters
validate_action = False
warnings = True

# Other parameters
rd_seed = 0
num_episodes = 1000
env_config = {  'M': 4, 
                'K': 4, 
                'T': 5, 
                'F': 10, 
                
                'min_sprice': 1,
                'max_sprice': 500, 
                'min_hprice': 1,
                'max_hprice': 500,

                'back_o_cost':20,
                
                'S': 5, 
                'LA_horizon': 4, 
                'lambda1': 0.5
            }
#################################   Environment's parameters   #################################

Next, the policy evaluation is defined. This function only takes one parameter, the number of episodes that will be runned of the environment. All the policies to be evaluated must be in the 'Policies.py' file. Any policy used must be able to receive the state, the additional information in _ and the environment as a parameters. Also, the policy function must return an action in the format defined in the Toying.ipynb.

In [3]:
'''

'''
def Policy_evaluation(num_episodes = 1000):
    
    rewards = {}
    states = {}
    actions = {}
    env = steroid_IRP(  horizon_type = horizon_type, look_ahead = look_ahead, historical_data = historical_data,
                        backorders = backorders, stochastic_parameters = stochastic_parameters, rd_seed = rd_seed,
                        env_config = env_config)

    policy = policies()

    for episode in range(num_episodes):

        state, _ = env.reset(return_state = True)
        print(env.d)
        done = False

        while not done:
            
            states[episode,env.t] = state
            action = policy.det_rolling_horizon(state, _, env)
            print(action)
            state, reward, done, _ = env.step(action, validate_action = validate_action, warnings = warnings)

            actions[episode,env.t] = action
            rewards[episode,env.t] = reward

    return rewards, states, actions

In [4]:
rewards, states, actions = Policy_evaluation(num_episodes = num_episodes)

{0: 16.5, 1: 23.5, 2: 12.0, 3: 0.0}
Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-19
[[], {(1, 0): 0.0, (1, 1): 0.0, (1, 2): -0.0, (1, 3): 0, (2, 0): 0.0, (2, 1): 0.0, (2, 2): 0, (2, 3): 0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0, (3, 3): 0}, {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (2, 0): 0.0, (2, 1): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0}]
Warning! Demand of product 0 was not fullfiled
Warning! Demand of product 1 was not fullfiled
Warning! Demand of product 2 was not fullfiled
[[], {(1, 0): 0.0, (1, 1): 0, (1, 2): 0, (1, 3): 0, (2, 0): 0.0, (2, 1): 0, (2, 2): 0.0, (2, 3): 0, (3, 0): 0.0, (3, 1): 0, (3, 2): 0.0, (3, 3): 0}, {(0, 0): 0.0, (0, 1): 0.0, (0, 2): 0.0, (0, 3): 0.0, (0, 4): 0.0, (1, 0): 0.0, (1, 1): 0.0, (1, 2): 0.0, (1, 3): 0.0, (1, 4): 0.0, (2, 0): 0.0, (2, 1): 0.0, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0, (3, 3): 0.0}]
Warning! 

/Users/juanbeta/My Drive/Investigación/SCA/SD-IB-IRP-PP/Environment/SD_IB_IRP_PPenv.py:42: DeprecationWarning: invalid escape sequence '\i'
  '''


KeyboardInterrupt: 